# Generate the Intron bed6

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(100)
%matplotlib inline

import pybedtools as pb
from tqdm import tqdm_notebook as tqdm

In [2]:
# f_gtf = 'ref/gencode.v29.annotation.gtf.gz'
# f_out = 'ref/gencode.v29.intron.bed.gz'
f_gtf = 'ref/gencode.vM20.annotation.gtf.gz'
f_out = 'ref/gencode.vM20.intron.bed.gz'
is_gencode = True

In [3]:
def get_gencode_gene(s_attr):
     return [x.split()[1].replace('"', '') 
             for x in s_attr.split('; ') if x.startswith('gene_name')][0]

In [4]:
df_gtf = pd.read_csv(f_gtf, header=None, sep='\t', comment='#')
df_gtf.columns = ['chr', 'source', 'feature', 'start', 'end', 'score', 
                  'strand', 'frame', 'attribute']
if is_gencode:
    df_gtf['symbol'] = df_gtf['attribute'].map(get_gencode_gene)
    
else:
    df_gtf['symbol'] = df_gtf.attribute.str.split('"').str.get(1)
df_gtf.head()

,chr,source,feature,start,end,score,strand,frame,attribute,symbol
0,chr1,HAVANA,gene,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693.1""; gene_type ""TEC...",4933401J01Rik
1,chr1,HAVANA,transcript,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693.1""; transcript_id ...",4933401J01Rik
2,chr1,HAVANA,exon,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693.1""; transcript_id ...",4933401J01Rik
3,chr1,ENSEMBL,gene,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842.1""; gene_type ""snR...",Gm26206
4,chr1,ENSEMBL,transcript,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842.1""; transcript_id ...",Gm26206


In [7]:
%%time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

l_g_without_intron = []
flag_intron = 0
for i,syn in enumerate(df_gtf.symbol.unique()):
    df_q = df_gtf.query('symbol==@syn')

    c = df_q.chr.unique()[0]
    s = df_q.start.min()
    e = df_q.end.max()
    syn = df_q.symbol.unique()[0]
    sc = '.'
    st = df_q.strand.unique()[0]
    
    df_q = df_q[df_q.feature != 'transcript']
    df_q = df_q[df_q.feature != 'gene'] 
    
    d = pd.DataFrame([[c,s,e,syn,sc,st]])
    g = pb.BedTool.from_dataframe(d)

    try:
        df_int_q = g.subtract(pb.BedTool.from_dataframe(df_q)).to_dataframe()
    except:
#         print(syn) # print genes without any intron.
        l_g_without_intron.append(syn)
    
    if 'df_int_q' in locals():
        if flag_intron == 0:
            df_intron = df_int_q
            flag_intron = 1

        elif df_int_q.name[0] == syn:
            df_intron = pd.concat([df_intron, df_int_q])

df_intron.to_csv(f_out, index=None, sep='\t', header=None)

CPU times: user 1h 23min 9s, sys: 44min 36s, total: 2h 7min 45s
Wall time: 2h 26min 50s


In [8]:
print(l_g_without_intron)

['4933401J01Rik', 'Gm26206', 'Gm18956', 'Gm37180', 'Gm37363', 'Gm37686', 'Gm37329', 'Gm7341', 'Gm38148', 'Gm10568', 'Gm38385', 'Gm27396', 'Gm37483', 'Gm22307', 'Gm38076', 'Gm7369', 'Gm6085', 'Gm6119', 'Gm25493', 'Gm2053', 'Gm6123', 'Gm37144', 'Gm6104', 'Gm37277', 'Gm17100', 'Gm37079', 'Gm17101', 'Gm37567', 'Gm38264', 'Gm36965', 'Gm22463', 'Gm37429', 'Npbwr1', 'Gm19214', '4732440D04Rik', 'Gm2147', 'Gm7417', 'Gm42492', 'Gm7449', 'Gm37275', 'Gm37225', 'Gm37489', 'Gm26983', 'Gm38372', 'Gm9826', 'Nras-ps2', 'Gm23274', 'Gm19002', 'Gm37791', 'Gm7470', 'Rps2-ps2', 'Gm38024', 'Gm16284', 'Gm38259', 'Gm23358', 'Gm37005', 'Gm15452', 'Gm36964', 'Gm24276', 'Gm6152', 'Gm7445', 'Gm7493', 'Gm37629', 'Gm38008', 'Gm37143', 'Gm18299', 'Gm7512', 'Gm24765', 'Rrs1', 'Gm6161', 'Gm6195', 'Gm22607', 'Snord87', 'E330040D14Rik', 'Gm24674', '1700047N06Rik', 'Gm26348', 'Gm37569', 'Gm38005', 'Gm15604', 'Gm25253', 'Gm15603', 'Gm37133', 'Gm5522', 'Gm28659', 'Gm22963', 'Gm24173', 'Gm28686', 'Gm37410', 'Gm38178', 'Gm292